In [ ]:
#!pip install kagglehub

#import pandas as pd
#import kagglehub

# Download dataset
#path = kagglehub.dataset_download("federalreserve/interest-rates")

# Load CSV
#df = pd.read_csv(f"{path}/index.csv")


# 3. Create a datetime column
#df['date'] = pd.to_datetime(dict(year=df.Year, month=df.Month, day=df.Day))
#df = df.sort_values("date").reset_index(drop=True)

#df.head() 

Defaulting to user installation because normal site-packages is not writeable


,Year,Month,Day,Federal Funds Target Rate,Federal Funds Upper Target,Federal Funds Lower Target,Effective Federal Funds Rate,Real GDP (Percent Change),Unemployment Rate,Inflation Rate,date
0,1954,7,1,NaN,NaN,NaN,0.80,4.6,5.8,NaN,1954-07-01
1,1954,8,1,NaN,NaN,NaN,1.22,NaN,6.0,NaN,1954-08-01
2,1954,9,1,NaN,NaN,NaN,1.06,NaN,6.1,NaN,1954-09-01
3,1954,10,1,NaN,NaN,NaN,0.85,8.0,5.7,NaN,1954-10-01
4,1954,11,1,NaN,NaN,NaN,0.83,NaN,5.3,NaN,1954-11-01


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import timedelta

def build_event_panel(fomc_dates, tickers, vol_window=5):
    rows = []

    start = fomc_dates.min() - timedelta(days=365)
    end = fomc_dates.max() + timedelta(days=10)

    # Market benchmark
    market_close = yf.download(
        "^GSPC", start=start, end=end, auto_adjust=True, progress=False
    )["Close"]

     # Must be series
    if isinstance(market_close, pd.DataFrame):
        market_close = market_close.iloc[:, 0]

    market_ret = market_close.pct_change()

    for ticker in tickers:
        print(f"Processing {ticker}...")

        asset_close = yf.download(
            ticker, start=start, end=end, auto_adjust=True, progress=False
        )["Close"]

        # Must be series
        if isinstance(asset_close, pd.DataFrame):
            asset_close = asset_close.iloc[:, 0]

        asset_ret = asset_close.pct_change()

        for event_date in fomc_dates:
            if event_date > asset_ret.index.max():
                continue

            loc = asset_ret.index.searchsorted(event_date)
            if loc >= len(asset_ret) or loc < vol_window:
                continue

            actual_date = asset_ret.index[loc]

            # --- scalar market return ---
            market_val = market_ret.loc[actual_date]
            if isinstance(market_val, pd.Series):
                market_val = market_val.iloc[0]

            asset_val = asset_ret.iloc[loc]
            if isinstance(asset_val, pd.Series):
                asset_val = asset_val.iloc[0]

            ar = asset_val - market_val

            # --- scalar volatility ---
            vol_window_ret = asset_ret.iloc[loc - vol_window:loc]
            vol = vol_window_ret.std()
            if isinstance(vol, pd.Series):
                vol = vol.iloc[0]

            rows.append({
                "event_date": actual_date,
                "ticker": ticker,
                "abnormal_return": float(ar),
                "volatility_5d": float(vol),
                "label": int(ar > 0)
            })

    return pd.DataFrame(rows)


In [33]:
event_df = build_event_panel(fomc_dates, tickers)

print(event_df.head())
print("\nMissing values:")
print(event_df.isna().sum())

print("\nData types:")
print(event_df.dtypes)

print("\nObservations per ticker:")
print(event_df.groupby("ticker").size())


Processing SPY...
Processing QQQ...
Processing IWM...
Processing XLF...
  event_date ticker  abnormal_return  volatility_5d  label
0 2015-12-16    SPY         0.000117       0.011856      1
1 2016-12-14    SPY        -0.000137       0.005306      0
2 2017-03-15    SPY         0.000279       0.002837      1
3 2017-06-14    SPY        -0.000272       0.002442      0
4 2017-09-20    SPY        -0.000274       0.000919      0

Missing values:
event_date         0
ticker             0
abnormal_return    0
volatility_5d      0
label              0
dtype: int64

Data types:
event_date         datetime64[ns]
ticker                     object
abnormal_return           float64
volatility_5d             float64
label                       int64
dtype: object

Observations per ticker:
ticker
IWM    37
QQQ    37
SPY    37
XLF    37
dtype: int64
